In [162]:
!pip install gdown

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [163]:
!pip install tf-models-official

In [164]:
import gdown
url = 'https://drive.google.com/file/d/1KCKfyIGbQi8a7bIYta3LM8dFStxVzVX-/view?usp=sharing'
output_path = '/kaggle/working/pre_data.zip'
gdown.download(url, output_path, quiet=False,fuzzy=True)

import zipfile
extract_dir = '/kaggle/working/'

with zipfile.ZipFile('/kaggle/working/pre_data.zip', 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

Downloading...
From (original): https://drive.google.com/uc?id=1KCKfyIGbQi8a7bIYta3LM8dFStxVzVX-
From (redirected): https://drive.google.com/uc?id=1KCKfyIGbQi8a7bIYta3LM8dFStxVzVX-&confirm=t&uuid=5a7da214-70c0-41fb-b5f8-2261fcbba20c
To: /kaggle/working/pre_data.zip
100%|██████████| 1.78G/1.78G [00:12<00:00, 142MB/s] 


In [165]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn import metrics

In [166]:
# Constants
IMG_SIZE = (64, 64)  # You can adjust this as needed
NUM_FRAMES = 20  # Number of frames per video
CATEGORIES = ["non shop lifters", "shop lifters"]
# CATEGORIES_ENC = [1, 0]

In [167]:
# Function to process videos into frames
def process_video(video_path, img_size=IMG_SIZE, num_frames=NUM_FRAMES):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while len(frames) < num_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, img_size)
        frames.append(frame)
    cap.release()
    
    # If fewer frames than required, pad with copies of the last frame
    while len(frames) < num_frames:
        frames.append(frames[-1])
    
    return np.array(frames)

In [168]:
# Load dataset
def load_dataset(base_dir):
    data = []
    labels = []
    
    for category in CATEGORIES:
        folder_path = os.path.join(base_dir, category)
        class_num = CATEGORIES.index(category)
        
        for video in os.listdir(folder_path):
            video_path = os.path.join(folder_path, video)
            video_frames = process_video(video_path)
            data.append(video_frames)
            labels.append(class_num)
    
    # Convert to NumPy arrays
    data = np.array(data)
    labels = to_categorical(labels, num_classes=len(CATEGORIES))
    
    return data, labels

# Example usage
data, labels = load_dataset("/kaggle/working/Shop DataSet")

In [169]:
data.shape

(855, 20, 64, 64, 3)

In [170]:
sum(labels == [0., 1.])

array([324, 324])

In [234]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42) 

In [235]:
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model

In [236]:
def build_classifier(batch_size, num_frames, resolution, backbone, num_classes):
    model = movinet_model.MovinetClassifier(
      backbone=backbone,
      num_classes=num_classes)
    model.build([batch_size, num_frames, resolution, resolution, 3])

    return model

In [237]:
backbone = movinet.Movinet(model_id='a0')

In [238]:
model = build_classifier(20, 20, 64, backbone, 2)

In [239]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [240]:
model.fit(X_train, y_train, epochs=25, batch_size=20, validation_split=0.2)

Epoch 1/25
28/28 [==============================] - 73s 883ms/step - loss: 0.9382 - accuracy: 0.5832 - val_loss: 0.6599 - val_accuracy: 0.6277
Epoch 2/25
28/28 [==============================] - 4s 147ms/step - loss: 0.6273 - accuracy: 0.6197 - val_loss: 0.6714 - val_accuracy: 0.6277
Epoch 3/25
28/28 [==============================] - 4s 144ms/step - loss: 0.5097 - accuracy: 0.6581 - val_loss: 0.6637 - val_accuracy: 0.6277
Epoch 4/25
28/28 [==============================] - 4s 145ms/step - loss: 0.2612 - accuracy: 0.9177 - val_loss: 0.7820 - val_accuracy: 0.3723
Epoch 5/25
28/28 [==============================] - 4s 145ms/step - loss: 0.2025 - accuracy: 0.9452 - val_loss: 1.8024 - val_accuracy: 0.3723
Epoch 6/25
28/28 [==============================] - 4s 145ms/step - loss: 0.1208 - accuracy: 0.9707 - val_loss: 10.1179 - val_accuracy: 0.3723
Epoch 7/25
28/28 [==============================] - 4s 145ms/step - loss: 0.1901 - accuracy: 0.9744 - val_loss: 10.1179 - val_accuracy: 0.3723
Epo

In [241]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'test loss: {test_loss}')
print(f'test accuracy: {test_acc}')


6/6 [==============================] - 6s 596ms/step - loss: 2.8032 - accuracy: 0.9591
test loss: 2.8031632900238037
test accuracy: 0.9590643048286438


In [242]:
model.save('Video clf.keras')

In [243]:
y_pred = model.predict(X_test)

6/6 [==============================] - 9s 586ms/step


In [244]:
y_test = y_test[:, 0]

In [245]:
y_test = np.expand_dims(y_test, axis=0).reshape(-1, 1)
y_test

array([[1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],

In [246]:
y_pred = model.predict(X_test)

6/6 [==============================] - 0s 56ms/step


In [247]:
y_pred = y_pred[:, 0]

In [248]:
y_pred = np.expand_dims(y_pred, axis=0).reshape(-1, 1) >= 0.5
y_pred

array([[ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [

In [249]:
import pandas as pd
cm = confusion_matrix(y_test, y_pred)

In [250]:
cm

array([[ 65,   0],
       [  0, 106]])

In [251]:
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
cm_display.plot(cmap=plt.cm.Blues)
plt.xticks([1, 0], ["non shop lifters", "shop lifters"])
plt.yticks([1, 0], ["non shop lifters", "shop lifters"])
# You can choose different color maps
plt.title('Confusion Matrix')
plt.show()

In [252]:
sum(y_pred - y_test)

array([0.])

In [253]:
sum(y_test)

array([106.])

In [254]:
sum(y_pred)

array([106])

In [255]:
y_test[0]

array([1.])